# Fasttext Supervised

https://fasttext.cc/docs/en/supervised-tutorial.html

## Prepare data

In [2]:
from hate.settings import corpora
train = corpora['train_es']
X_train, y_train = list(train.X()), list(train.y())
dev = corpora['dev_es']
X_dev, y_dev = list(dev.X()), list(dev.y())

In [5]:
f = open('fasttext_train_es.txt', 'w')
for x, y in zip(X_train, y_train):
    f.write('__label__{} {}\n'.format(y, x))
f.close()

In [6]:
f = open('fasttext_dev_es.txt', 'w')
for x, y in zip(X_dev, y_dev):
    f.write('__label__{} {}\n'.format(y, x))
f.close()

## Train from scratch

In [17]:
import fastText

In [2]:
fastText.train_supervised?

Signature:
fastText.train_supervised(
    ['input', 'lr=0.1', 'dim=100', 'ws=5', 'epoch=5', 'minCount=1', 'minCountLabel=0', 'minn=0', 'maxn=0', 'neg=5', 'wordNgrams=1', "loss='softmax'", 'bucket=2000000', 'thread=19', 'lrUpdateRate=100', 't=0.0001', "label='__label__'", 'verbose=2', "pretrainedVectors=''"],
)
Docstring:
Train a supervised model and return a model object.

input must be a filepath. The input text does not need to be tokenized
as per the tokenize function, but it must be preprocessed and encoded
as UTF-8. You might want to consult standard preprocessing scripts such
as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html

The input file must must contain at least one label per line. For an
example consult the example datasets which are part of the fastText
repository such as the dataset pulled by classification-example.sh.
File:      ~/.virtualenvs/pln/lib/python3.7/site-packages/fastText/FastText.py
Type:      function


In [56]:
model = fastText.train_supervised('fasttext_train_es.txt', lr=0.1, epoch=25, wordNgrams=2)

In [40]:
model.save_model('SUPERVISED.bin')

## Test

In [57]:
model.test('fasttext_dev_es.txt')
# model.test?

(500, 0.77, 0.77)

## Train from Standard Pretrained

In [66]:
model = fastText.train_supervised(
    'fasttext_train_es.txt',
    lr=0.1,
    epoch=25,
    wordNgrams=2,
    dim=300,
    #pretrainedVectors='/users/jsanchez/francolq/hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin'
    pretrainedVectors='cc.es.300.vec'
)

In [67]:
model.test('fasttext_dev_es.txt')

(500, 0.768, 0.768)

In [ ]:
model.save_model(...)

## Train from Our Pretrained

Here improves a bit.

In [31]:
model = fastText.train_supervised(
    'fasttext_train_es.txt',
    lr=0.5,
    epoch=20,
    wordNgrams=2,
    dim=50,
    pretrainedVectors='fixed_fasttext_train_es.vec'
)

In [32]:
model.test('fasttext_dev_es.txt')

(500, 0.778, 0.778)

## Train from our pretrained with proper preprocessing

Improves more!

In [47]:
from hate.tokenizer import Tokenizer
tkn = Tokenizer(lang='es', lem=False, neg=True, rdup=True)
X_train_tokens = [tkn(x) for x in X_train]
X_dev_tokens = [tkn(x) for x in X_dev]

In [12]:
import pandas as pd
pd.options.display.max_colwidth = 0
df = pd.DataFrame([(x, x2) for x, x2 in zip(X_train, X_train_tokens)])
df[:10]

,0,1
0,Easyjet quiere duplicar el número de mujeres piloto' Verás tú para aparcar el avión.. http://t.co/46NuLkm09x,"[easyjet, quiere, duplicar, número, mujeres, piloto, verás, aparcar, avión, url]"
1,El gobierno debe crear un control estricto de inmigración en las zonas fronterizas con Colombia por q después del 20-8querrán venir en masa,"[gobierno, debe, crear, control, estricto, inmigración, zonas, fronterizas, colombia, q, después, querrán, venir, masa]"
2,Yo veo a mujeres destruidas por acoso laboral y callejero. Otras con depresión debido a violación sexual o maltrato físico. Y conocí a varias que se suicidaron por este tipo de comportamientos machistas. Vas a seguir con tu show pobre? https://t.co/JE6MEgvIh4,"[veo, mujeres, destruidas, acoso, laboral, callejero, depresión, debido, violación, sexual, maltrato, físico, conocí, varias, suicidaron, tipo, comportamientos, machistas, vas, seguir, show, pobre, url]"
3,"— Yo soy respetuoso con los demás, sólamente les recuerdo lo que son. Y si son escoria no es culpa mía. ... — ¡Claro que lo sé! ¿¡Por quién me tomas!?","[—, respetuoso, demás, sólamente, recuerdo, si, escoria, no, NOT_culpa, —, claro, sé, quién, tomas]"
4,Antonio Caballero y como ser de mal gusto e ignorante a la vez. Claro que no es lo mismo acoso sexual y violación pero hay varias conductas que componen la violencia sexual.,"[antonio, caballero, ser, mal, gusto, ignorante, vez, claro, no, NOT_mismo, NOT_acoso, NOT_sexual, violación, varias, conductas, componen, violencia, sexual]"
5,@Zurine3 Cuánto hijo de puta sobra en el mundo...,"[@user, cuánto, hijo, puta, sobra, mundo]"
6,Pablo Casado debería estudiar un máster en inmigración https://t.co/Pcz6GPJ6dG,"[pablo, casado, debería, estudiar, máster, inmigración, url]"
7,vos me haces eso te pego una patada en la cara https://t.co/2drqUu5Rwc,"[vos, haces, pego, patada, cara, url]"
8,#Noticiasdelalinea: Detienen en Guatapanal motocicleta y nacional haitiano indocumentado https://t.co/qs9MTKddqK,"[#noticiasdelalinea, detienen, guatapanal, motocicleta, nacional, haitiano, indocumentado, url]"
9,@MaivePerez Llora te lo.mereces por zorra,"[@user, llora, lo.mereces, zorra]"


In [48]:
f = open('fasttext_sentiment_train_es.txt', 'w')
for x, y in zip(X_train_tokens, y_train):
    f.write('__label__{} '.format(y) + ' '.join(x) + '\n')
f.close()
f = open('fasttext_sentiment_dev_es.txt', 'w')
for x, y in zip(X_dev_tokens, y_dev):
    f.write('__label__{} '.format(y) + ' '.join(x) + '\n')
f.close()

In [ ]:
$ ./fasttext print-word-vectors ../hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin < ../hatEval/notebooks/fasttext_sentiment_train_es.txt > ../hatEval/notebooks/fasttext_sentiment_train_es.vec
$ wc -l fasttext_sentiment_train_es.vec
59148 fasttext_sentiment_train_es.vec
$ echo '59148 50' | cat - fasttext_sentiment_train_es.vec > fixed_fasttext_sentiment_train_es.vec


In [36]:
model = fastText.train_supervised(
    'fasttext_sentiment_train_es.txt',
    lr=0.1,
    epoch=20,
    wordNgrams=2,
    dim=50,
    pretrainedVectors='fixed_fasttext_sentiment_train_es.vec'
)

In [55]:
model = fastText.train_supervised(
    'fasttext_sentiment_train_es.txt',
    lr=0.5,
    epoch=20,
    wordNgrams=2,
    dim=50,
    pretrainedVectors='fixed_fasttext_sentiment_train_es.vec'
)

In [56]:
model.test('fasttext_sentiment_dev_es.txt')

(500, 0.792, 0.792)

In [57]:
model.save_model('SUPERVISED_SENTIMENT.bin')

## Test our original embeddings with no supervision

You stupid it was never trained it can't be tested

In [13]:
import fastText
filename = '/users/jsanchez/francolq/hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin'
model = fastText.load_model(filename)

In [ ]:
model.test()

In [ ]:
$ ./fasttext test ../hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin ../hatEval/notebooks/fasttext_sentiment_dev_es.txt 
N	0
P@1	-nan
R@1	-nan
$ ./fasttext test ../hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin ../hatEval/notebooks/fasttext_dev_es.txt 
N	0
P@1	-nan
R@1	-nan


## Test in Pipeline

Really sad.

In [5]:
from hate.classifier import HateClassifier

# filename = 'SUPERVISED.bin'
filename = 'SUPERVISED_SENTIMENT.bin'
clf = HateClassifier(
    lang='es',
    bow=True,
    bow_params={
        'binary': True,
        'ngram_range': (1, 2),
    },
    boc=True,
    boc_params={
        'analyzer': 'char',
        'binary': True,
        'ngram_range': (1, 5),
    },
    #emb='fasttext',
    emb='wfasttext',
    emb_params={
        'file': filename,
        #'tokenizer': None,  # for SUPERVISED.bin
    },
    #clf='maxent'
)
clf.fit(X_train, y_train)
clf.eval(X_dev, y_dev)

accuracy	0.78

              precision    recall  f1-score   support

           0       0.84      0.75      0.79       278
           1       0.72      0.82      0.77       222

   micro avg       0.78      0.78      0.78       500
   macro avg       0.78      0.79      0.78       500
weighted avg       0.79      0.78      0.78       500

[[208  70]
 [ 39 183]]


In [10]:
# ONLY FASTTEXT
from hate.classifier import HateClassifier

#filename = 'SUPERVISED.bin'
filename = 'SUPERVISED_SENTIMENT.bin'
clf = HateClassifier(
    lang='es',
    bow=False,
    boc=False,
    emb='fasttext',
    #emb='wfasttext',
    emb_params={
        'file': filename,
        #'tokenizer': None,
    },
    clf='svm'
)
clf.fit(X_train, y_train)
clf.eval(X_dev, y_dev)

accuracy	0.79

              precision    recall  f1-score   support

           0       0.84      0.78      0.81       278
           1       0.75      0.81      0.78       222

   micro avg       0.79      0.79      0.79       500
   macro avg       0.79      0.80      0.79       500
weighted avg       0.80      0.79      0.79       500

[[217  61]
 [ 42 180]]
